In [30]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
EPS = 1e-15
from torch.nn import Parameter
import pandas as pd
import numpy as np 
device = 'cpu'
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
beta = 4.4
gama = 0.5

def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    print("预测精确率： ",average_precision_score(true_edges, predict_edges))
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("预测准确率： ",accuracy_score(true_edges, predict_edges))
    #print("预测精确率： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("召回率： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))
    
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, ne, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        NE = ne
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, cc):
        CC = cc
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=0.4, momentum=0.9)
        
        #self.loss_function = torch.nn.L1Loss(reduction='sum')
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, cc, edge_train, edge_test):
        CC = cc
        GP = g
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach()
    
####################################### 以最大化预测效果为目标的类型质心更新 ########################################
class ClusteringLayer(nn.Module):
    """给定节点嵌入，更新类簇质心"""
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, x):
        NE = x
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(ne, ini_cc, g, edge_train, edge_test, num_epochs=5):        
    NE = ne
    CC = ini_cc
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=0.4, momentum=0.9)
    
    #loss_function = torch.nn.L1Loss(reduction='sum')
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach()

In [31]:
def get_graph(filename_adj, nodes_number):
    raw_edges = pd.read_csv(filename_adj,header=None,sep=' ') - 1
    
    drop_self_loop = raw_edges[raw_edges[0] != raw_edges[1]]
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(drop_self_loop.shape[0]):
        graph_np[drop_self_loop.iloc[i,0], drop_self_loop.iloc[i,1]]=1
        graph_np[drop_self_loop.iloc[i,1], drop_self_loop.iloc[i,0]]=1
    
    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    positive_index = np.random.choice(range(posi_edge.shape[0]),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge.shape[0]), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [32]:
filename = "data/ia-email-univ.mtx"
nodes_number = 1133
train_edge, test_edge, train_mask, test_mask = get_graph(filename, nodes_number)

In [33]:
len(train_edge),len(test_edge)

(24525, 1092)

In [34]:
embedding_dim = 12
n_clusters = 24
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

In [35]:
raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(200):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
预测精确率：  0.6240792269118003
AUC SCORE:  0.6322136349608877
预测精确率：  0.6515035797358935
AUC SCORE:  0.6507097908196809
######################### 模块循环 ： 1 ##########################
预测精确率：  0.6845155043385068
AUC SCORE:  0.6647747856539065
预测精确率：  0.691098386556835
AUC SCORE:  0.6797286962122127
######################### 模块循环 ： 2 ##########################
预测精确率：  0.7334520978978905
AUC SCORE:  0.7187235841081994
预测精确率：  0.751902453132533
AUC SCORE:  0.7417984945457473
######################### 模块循环 ： 3 ##########################
预测精确率：  0.7698270143826192
AUC SCORE:  0.7566081659488253
预测精确率：  0.7721789791737117
AUC SCORE:  0.7573293617249661
######################### 模块循环 ： 4 ##########################
预测精确率：  0.7834782296749283
AUC SCORE:  0.7623777321579518
预测精确率：  0.7909136722270401
AUC SCORE:  0.7700257617840035
######################### 模块循环 ： 5 ##########################
预测精确率：  0.8030644021347007
AUC SCORE:  0.779948073

预测精确率：  0.8937904664886835
AUC SCORE:  0.88003663003663
预测精确率：  0.8930449019445504
AUC SCORE:  0.8788961343906398
######################### 模块循环 ： 47 ##########################
预测精确率：  0.8934617590907892
AUC SCORE:  0.8801137812126822
预测精确率：  0.8930649169699152
AUC SCORE:  0.8796542285553275
######################### 模块循环 ： 48 ##########################
预测精确率：  0.8933696128979318
AUC SCORE:  0.8802043499845698
预测精确率：  0.8931271895607064
AUC SCORE:  0.8798118853063909
######################### 模块循环 ： 49 ##########################
预测精确率：  0.8936710117243597
AUC SCORE:  0.8806940922325538
预测精确率：  0.8935351338203273
AUC SCORE:  0.8804425123106442
######################### 模块循环 ： 50 ##########################
预测精确率：  0.8937440019159467
AUC SCORE:  0.8810764937138564
预测精确率：  0.8935138417834898
AUC SCORE:  0.8807913698023587
######################### 模块循环 ： 51 ##########################
预测精确率：  0.8940107696182261
AUC SCORE:  0.8814387688014061
预测精确率：  0.8938760688106345
AUC SCORE:  0.88132471

预测精确率：  0.896840959650791
AUC SCORE:  0.88916394960351
预测精确率：  0.896715207694162
AUC SCORE:  0.8884662346200808
######################### 模块循环 ： 93 ##########################
预测精确率：  0.8974620211855543
AUC SCORE:  0.8890901528264166
预测精确率：  0.8971085852112255
AUC SCORE:  0.8883756658481933
######################### 模块循环 ： 94 ##########################
预测精确率：  0.8967621234792966
AUC SCORE:  0.8886205369721853
预测精确率：  0.8962204992254308
AUC SCORE:  0.8882045915012947
######################### 模块循环 ： 95 ##########################
预测精确率：  0.8965799586136651
AUC SCORE:  0.8888654080961773
预测精确率：  0.8952938880861836
AUC SCORE:  0.8874901045230716
######################### 模块循环 ： 96 ##########################
预测精确率：  0.8966828664581017
AUC SCORE:  0.8889023064847241
预测精确率：  0.8961464733694641
AUC SCORE:  0.8882280722940064
######################### 模块循环 ： 97 ##########################
预测精确率：  0.8969031197199915
AUC SCORE:  0.8888586992982597
预测精确率：  0.8967063475827771
AUC SCORE:  0.8884897154

预测精确率：  0.8964685564547452
AUC SCORE:  0.8905929235599566
预测精确率：  0.8961409521586923
AUC SCORE:  0.8901602060942719
######################### 模块循环 ： 139 ##########################
预测精确率：  0.8960544158716195
AUC SCORE:  0.8908411490829073
预测精确率：  0.8950856857273377
AUC SCORE:  0.8897107166337936
######################### 模块循环 ： 140 ##########################
预测精确率：  0.8962453163728719
AUC SCORE:  0.8906063411557916
预测精确率：  0.8958505422880998
AUC SCORE:  0.890046156529673
######################### 模块循环 ： 141 ##########################
预测精确率：  0.8961895573120068
AUC SCORE:  0.8910055146318883
预测精确率：  0.8948810932761362
AUC SCORE:  0.8892008479920568
######################### 模块循环 ： 142 ##########################
预测精确率：  0.8960737596097778
AUC SCORE:  0.8904419756068107
预测精确率：  0.8960973758461201
AUC SCORE:  0.8903950140213874
######################### 模块循环 ： 143 ##########################
预测精确率：  0.8963273393148655
AUC SCORE:  0.8910591850152291
预测精确率：  0.8946746551356686
AUC SCORE:  0.88

预测精确率：  0.8961395018959625
AUC SCORE:  0.8902876732547063
预测精确率：  0.8958269760057238
AUC SCORE:  0.8899807457499765
######################### 模块循环 ： 185 ##########################
预测精确率：  0.8964367959692359
AUC SCORE:  0.8904486844047284
预测精确率：  0.8957970241689449
AUC SCORE:  0.8897442606233814
######################### 模块循环 ： 186 ##########################
预测精确率：  0.8962441370952071
AUC SCORE:  0.8903514068349233
预测精确率：  0.8959748582397431
AUC SCORE:  0.8900059037421675
######################### 模块循环 ： 187 ##########################
预测精确率：  0.8965757516537821
AUC SCORE:  0.8905023547880689
预测精确率：  0.8957834387827366
AUC SCORE:  0.8897643870171343
######################### 模块循环 ： 188 ##########################
预测精确率：  0.8963347918356928
AUC SCORE:  0.8904151404151404
预测精确率：  0.8959912821526606
AUC SCORE:  0.889979068550497
######################### 模块循环 ： 189 ##########################
预测精确率：  0.8965258458521612
AUC SCORE:  0.8905291899797394
预测精确率：  0.8957635371310345
AUC SCORE:  0.88